In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
test_id=test.ID
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
y=train.target
y=np.log(y+1)
y.head()

In [ ]:

train=train.drop(['ID'],axis=1)
train=train.drop(['target'],axis=1)
test=test.drop(['ID'],axis=1)

In [ ]:
unique_df=train.nunique().reset_index()
unique_df.columns=["column","unique_count"]
constant_df=unique_df[unique_df["unique_count"]==1]
constant_df.shape

In [ ]:
train=train.drop(constant_df['column'],axis=1)
test=test.drop(constant_df['column'],axis=1)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
total=(train==0).sum().sort_values(ascending=True)
percent=((train==0).sum()/((train==0).count())*100).sort_values(ascending=False)
train_data=pd.concat([total,percent],axis=1,keys=['total','percent'],sort=False)
train_data.head(50)

In [ ]:
cols_used=train_data[train_data.percent<80]
cols_used.shape

In [ ]:
cols_used=cols_used.index
cols_used.shape

In [ ]:
cols_used

In [ ]:
train=train[cols_used]
test=test[cols_used]
print(train.shape)
print(test.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train,y,random_state=0)

In [ ]:
def rmsle(x,y):
    return np.sqrt(np.square(np.log(x-1)-np.log(y-1)).mean())

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#model=RandomForestRegressor()


In [ ]:
'''from sklearn.model_selection import GridSearchCV
max_features=[0.5,0.75,0.85,0.95]
min_samples_leaf=np.arange(1,15)
min_samples_split=np.arange(2,15)
n_estimators=np.arange(1,11)*10
params={'max_features':max_features,'min_samples_leaf':min_samples_leaf,'min_samples_split':min_samples_split,'n_estimators':n_estimators}'''

In [ ]:
'''grid_search=GridSearchCV(model,param_grid=params)
grid_search.fit(train,y)'''

In [ ]:
model=RandomForestRegressor(max_features=0.75,n_estimators=100,min_samples_leaf=11,min_samples_split=13)
model.fit(train,y)

In [ ]:
print(model.score(train,y))
#print(model.score(X_test,y_test))

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
print(rmsle(np.exp(y_pred)-1,np.exp(y_test)-1))

In [ ]:
out_test=model.predict(test)

In [ ]:
out_test=np.exp(out_test)-1

In [ ]:
out_df=pd.DataFrame(out_test)

In [ ]:
out_df.columns = ['target']
out_df.insert(0, 'ID', test_id)
#out_df

In [ ]:
out_df.to_csv('santander_submission.csv',index=False)

In [ ]:
#test_id.iloc[23042]

In [ ]:
#out_df.iloc[23029]